In [4]:
import numpy as np
import glob
import os,sys
import pandas as pd
import re
curr_path = sys.path[0]
os.chdir(curr_path+"/../Analytic_Flux/")
curr_path = os.getcwd()+"/"
OutFolder = curr_path+"ProcaEnDenData/"
InFolder = curr_path+"CSVData/"

if not os.path.exists(OutFolder):
    os.mkdir(OutFolder)
def contains_str(str, regex):
    _regex = re.compile(regex)
    return bool(_regex.search(str))

In [5]:
allfiles = glob.glob(InFolder+"*")
allfilenames = np.array(list(map(os.path.basename, allfiles)))
fun = lambda str: contains_str(str, "EnergyDensityCOORDS*")
COORDS_filenames = allfilenames[list(map(fun, allfilenames))]
fun = lambda str: contains_str(str, "EnergyDensityVALUES*")
VALUES_filenames = allfilenames[list(map(fun, allfilenames))]


In [3]:
coordvaluepair = []
for valuestr in VALUES_filenames:
    try:
        data_name = valuestr[20:]
        truth = [i[20:]==valuestr[20:] for i in COORDS_filenames]
        coordstr = COORDS_filenames[truth][0]
        coord_data = pd.read_csv(InFolder+coordstr, header=None, na_values="None")
        value_data = pd.read_csv(InFolder+valuestr, header=None, na_values="None")
        rdata = coord_data[0].dropna().values
        thdata = coord_data[1].dropna().values
        valuedata = value_data.values

        out_filename = OutFolder+data_name[:-4]
        np.savez(out_filename, RadialData=rdata, ThetaData=thdata, EnergyData=valuedata)
    except KeyError:
        print("Error at: \n\t {0} \n\t {1}\n skipping \n".format(valuestr,coordstr))

In [13]:
alldata = np.array(glob.glob(OutFolder+"/*"))

In [14]:
alldata
modeovertonedata = alldata[[i[-21:-4]=="Mode_"+str(1)+"_Overtone_"+str(0) for i in alldata]]
alpha_regex= re.compile("Alpha_\d+_\d+")
alpha_matches = np.array([alpha_regex.findall(string) for string in modeovertonedata])
value_regex = re.compile("\d+")
numdoms = [list(map(float, value_regex.findall(st[0]))) for st in alpha_matches]
alphas=np.array([i[0]/i[1] for i in numdoms])
tmp =np.abs(0.5-alphas)
index = np.where(tmp==tmp.min())
result = np.array(modeovertonedata)[index][0]

In [15]:
data=np.load(result)

In [16]:
data["EnergyData"].shape

(287, 100)

In [17]:
data["RadialData"].shape

(287,)

In [18]:
data["ThetaData"].shape

(100,)

In [19]:
alphas.min()

0.02